<a href="https://colab.research.google.com/github/Patcharaporn2093/Data-ANA-ML/blob/main/Pokemon_Move_MLPClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
!pip install nltk # it's the Natural Language Toolkit, a library for working with human language data in Python.
!pip install contractions #This package provides functionality to expand English language contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.2 MB/s eta 0:00:00


In [ ]:
#import data

!gdown 1CN8q9cmFy9ozhuAqyHegnim_8GSRHk5O
!gdown 1FHJfxCjC23B7skIBxye6_piQyO2sqFKV

Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/gdown/cli.py", line 151, in main
    filename = download(
  File "/usr/local/lib/python3.10/dist-packages/gdown/download.py", line 203, in download
    filename_from_url = m.groups()[0]
AttributeError: 'NoneType' object has no attribute 'groups'
Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/gdown/cli.py", line 151, in main
    filename = download(
  File "/usr/local/lib/python3.10/dist-packages/gdown/download.py", line 203, in download
    filename_from_url = m.groups()[0]
AttributeError: 'NoneType' object has no attribute 'groups'


In [ ]:
import numpy as np
import pandas as pd

datatrain = pd.read_csv('/content/train.csv')
datatrain.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '/content/train.csv'

In [ ]:
datatrain.info()

In [ ]:
datatest = pd.read_csv('test.csv')
datatest.head(10)

In [ ]:
datatest.info()

# Clean text

In [ ]:
import contractions
import re

def preprocess_flavor_text(text):
    text = text.lower() #Converts all characters in the text to lowercase.
    text = contractions.fix(text) #expand English language contractions. For example, it turns "don't" into "do not."
    text = re.sub(r'[^\w\s]', '', text) #Removes non-alphanumeric characters and underscores from the text using a regular expression.
    text = text.replace("โ€s", "") #Replaces a specific substring ("โ€s") with an empty string.
    text = re.sub(r'[^\x00-\x7F]+', '', text) #Removes non-ASCII characters from the text using a regular expression.
    text = re.sub(r'\s+', ' ', text) #Replaces consecutive whitespace characters with a single space.

    return text

datatrain["cleaned_text"] = datatrain["flavor_text"].apply(preprocess_flavor_text)
datatrain

# Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Splits the data into training and testing sets using 80% for training and 20% for testing.
# The cleaned_text column is used as the input features (X_train and X_test), and the damageclass column is used as the target variable (y_train and y_test).
X_train, X_test, y_train, y_test = train_test_split(
    datatrain['cleaned_text'], datatrain['damageclass'], test_size=0.2, random_state=42)

# Constructs a machine learning pipeline using two components: a TfidfVectorizer for extracting text features and an MLPClassifier (neural network).
# The TfidfVectorizer is configured with sublinear term frequency scaling and L2 normalization, while the MLPClassifier has an activation function of 'relu' and a maximum of 500 iterations.
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(sublinear_tf=True, smooth_idf=True, norm='l2')),
    ('clf', MLPClassifier(activation='relu', max_iter=500))])

# Grid of hyperparameters for model tuning
param_grid = {
    # Varying the maximum number of features for the TfidfVectorizer
    'tfidf__max_features': [5000, 10000, 15000],
    # This parameter controls the range of n-grams used by the TfidfVectorizer. The values [(1, 1), (1, 2)] indicate that you are considering both unigrams (single words) and bigrams (pairs of consecutive words) as features.
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    # Exploring different architectures with varying hidden layer sizes for MLPClassifier
    'clf__hidden_layer_sizes': [(50, 50), (100, 100), (50,)],
    # This is the L2 regularization term in the MLPClassifier. It helps prevent overfitting by penalizing overweight individuals.
    'clf__alpha': [0.0001, 0.001, 0.01],
    # This parameter sets the maximum number of iterations for the optimization algorithm in the MLPClassifier.
    'clf__max_iter': [300, 500, 700],
    # Varying the beta_1 parameter in the Adam optimizer used by MLPClassifier
    'clf__beta_1': [0.9, 0.95, 0.99],
    # Varying the beta_2 parameter in the Adam optimizer used by MLPClassifier
    'clf__beta_2': [0.999, 0.9999],
}

# Utilizes GridSearchCV to perform hyperparameter tuning by searching through the defined parameter grid. Cross-validation with 3 folds is used to evaluate different combinations of hyperparameters.
grid_search = GridSearchCV(pipeline, param_grid, cv=3)
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best Parameters:", grid_search.best_params_)

# Uses the best model to make predictions on the testing set.
y_pred = grid_search.predict(X_test)

# Evaluate the model
print("Classification Report:\n", classification_report(y_test, y_pred))

# Apply the model to the test data
datatest["damageclass"] = grid_search.predict(datatest['flavor_text'])
datatest.head(10)

Best Parameters: {'clf__alpha': 0.0001, 'clf__beta_1': 0.99, 'clf__beta_2': 0.999, 'clf__hidden_layer_sizes': (100, 100), 'clf__max_iter': 300, 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 2)}
Classification Report:
               precision    recall  f1-score   support

    physical       0.76      0.79      0.78        53
     special       0.70      0.52      0.59        31
      status       0.80      0.94      0.87        35

    accuracy                           0.76       119
   macro avg       0.75      0.75      0.75       119
weighted avg       0.76      0.76      0.76       119



,move_name,flavor_text,damageclass
0,pound,The target is physically pounded with a long t...,physical
1,comet-punch,The target is hit with a flurry of punches tha...,physical
2,fire-punch,The target is punched with a fiery fist. This ...,physical
3,ice-punch,The target is punched with an icy fist. This m...,physical
4,swords-dance,A frenetic dance to uplift the fighting spirit...,status
5,gust,A gust of wind is whipped up by wings and laun...,special
6,wing-attack,"The target is struck with large, imposing wing...",physical
7,fly,The user flies up into the sky and then\nstrik...,physical
8,body-slam,The user drops onto the target with its full b...,physical
9,double-edge,"A reckless, life-risking tackle. This also dam...",physical


In [ ]:
datatest[["move_name", "damageclass"]].to_csv("Testdata16.csv",index=False)